## 导入工具包

In [1]:
import os
import shutil
import random
import pandas as pd

## 获得所有类别名称

In [2]:
# 指定数据集路径
dataset_path = 'jiyouguan'

In [3]:
dataset_name = dataset_path.split('_')[0]
print('数据集', dataset_name)

数据集 jiyouguan


In [4]:
classes = os.listdir(dataset_path)

In [5]:
len(classes)

4

In [6]:
classes

['body_broken', 'normal', 'side_broken', 'top_broken']

## 创建训练集文件夹和测试集文件夹

In [7]:
# 创建 train 文件夹
os.mkdir(os.path.join(dataset_path, 'train'))

# 创建 test 文件夹
os.mkdir(os.path.join(dataset_path, 'val'))

# 在 train 和 test 文件夹中创建各类别子文件夹
for youguan in classes:
    os.mkdir(os.path.join(dataset_path, 'train', youguan))
    os.mkdir(os.path.join(dataset_path, 'val', youguan))

## 划分训练集、测试集，移动文件

In [8]:
test_frac = 0.2  # 测试集比例
random.seed(123) # 随机数种子，便于复现

In [9]:
df = pd.DataFrame()

print('{:^18} {:^18} {:^18}'.format('类别', '训练集数据个数', '测试集数据个数'))

for youguan in classes: # 遍历每个类别

    # 读取该类别的所有图像文件名
    old_dir = os.path.join(dataset_path, youguan)
    images_filename = os.listdir(old_dir)
    random.shuffle(images_filename) # 随机打乱

    # 划分训练集和测试集
    testset_numer = int(len(images_filename) * test_frac) # 测试集图像个数
    testset_images = images_filename[:testset_numer]      # 获取拟移动至 test 目录的测试集图像文件名
    trainset_images = images_filename[testset_numer:]     # 获取拟移动至 train 目录的训练集图像文件名

    # 移动图像至 test 目录
    for image in testset_images:
        old_img_path = os.path.join(dataset_path, youguan, image)         # 获取原始文件路径
        new_test_path = os.path.join(dataset_path, 'val', youguan, image) # 获取 test 目录的新文件路径
        shutil.move(old_img_path, new_test_path) # 移动文件

    # 移动图像至 train 目录
    for image in trainset_images:
        old_img_path = os.path.join(dataset_path, youguan, image)           # 获取原始文件路径
        new_train_path = os.path.join(dataset_path, 'train',youguan, image) # 获取 train 目录的新文件路径
        shutil.move(old_img_path, new_train_path) # 移动文件
    
    # 删除旧文件夹
    assert len(os.listdir(old_dir)) == 0 # 确保旧文件夹中的所有图像都被移动走
    shutil.rmtree(old_dir) # 删除文件夹
    
    # 工整地输出每一类别的数据个数
    print('{:^18} {:^18} {:^18}'.format(youguan, len(trainset_images), len(testset_images)))
    
    # 保存到表格中
    df = df.append({'class':youguan, 'trainset':len(trainset_images), 'testset':len(testset_images)}, ignore_index=True)

# 重命名数据集文件夹
shutil.move(dataset_path, dataset_name+'_split')

# 数据集各类别数量统计表格，导出为 csv 文件
df['total'] = df['trainset'] + df['testset']
df.to_csv('数据量统计.csv', index=False)

        类别              训练集数据个数            测试集数据个数      
   body_broken            441                110        
      normal              717                179        
   side_broken            121                 30        
    top_broken            137                 34        


In [10]:
df

,class,trainset,testset,total
0,body_broken,441.0,110.0,551.0
1,normal,717.0,179.0,896.0
2,side_broken,121.0,30.0,151.0
3,top_broken,137.0,34.0,171.0


## 查看文件目录结构

In [11]:
!sudo snap install tree

tree 1.8.0+pkg-3fd6 已从 林博仁(Buo-ren, Lin) (brlin) 安装      |07ms


In [13]:
!tree jiyouguan_split -L 2

jiyouguan_split
├── train
│   ├── body_broken
│   ├── normal
│   ├── side_broken
│   └── top_broken
└── val
    ├── body_broken
    ├── normal
    ├── side_broken
    └── top_broken

10 directories, 0 files
